## Project Structure

```arduino
ticket-reporter/
│
├── app/
│   ├── __init__.py
│   ├── models.py
│   ├── views.py
│   ├── forms.py
│   ├── data/
│   │   └── sqlite/
│   │       └── app.db
│   ├── templates/
│   │   ├── base.html
│   │   ├── home.html
│   │   ├── task.html
│   │   ├── ticket.html
│   │   ├── project.html
│   │   └── about.html
│   └── static/
│       ├── css/
│       │   └── styles.css
│       ├── js/
│       │   └── scripts.js
│       └── favicon.ico
├── db_init.py
├── run.py
├── config.py
├── runner.ipynb
└── requirements.txt
```

## Verify Test Data

In [ ]:
import sqlite3
import os
import sys
import pandas as pd

from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import OperationalError
from sqlalchemy.orm import sessionmaker

# 添加项目根目录到 sys.path
project_root = 'd:\\Projects\\Py-Projects\\Flask-Projects\\ticket-reporter'
sys.path.append(project_root)

from app import create_app, db
from app.models import *

# 创建 Flask 应用实例
app = create_app()

# 提取数据库路径
db_uri = app.config['SQLALCHEMY_DATABASE_URI']
if db_uri.startswith('sqlite:///'):
    db_path = db_uri[len('sqlite:///'):]
elif db_uri.startswith('sqlite://'):
    db_path = db_uri[len('sqlite://'):]
else:
    raise ValueError("Unsupported database URI scheme")

# 在 Jupyter Notebook 中手动指定路径
db_path = os.path.join(os.getcwd(), 'instance', 'report.db')

# 检查数据库文件是否存在
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file not found: {db_path}")

# 打印数据库路径以进行调试
print(f"Database path: {db_path}")

# 使用 SQLAlchemy 的连接引擎
engine = create_engine(f'sqlite:///{db_path}', echo=True)

# 列出数据库中实际存在的表
inspector = inspect(engine)
existing_tables = inspector.get_table_names()
print("Existing tables in the database:")
print(existing_tables)


from app.utilities import model_to_dataframe

model_map = {
    'client_table': Client,
    'device_table': Device,
    'site_table': Site,
    'ticket_table': Ticket,
    'task_table': Task,
    'issue_table': Issue,
    'resolution_table': Resolution
}

with app.app_context():
    try:
        # 读取表格内容为所有实例的 list 并转换为 DataFrame
        client_df = model_to_dataframe(Client)
        print(f"\n{len(client_df)} clients")
        print(client_df.columns)
        device_df = model_to_dataframe(Device)
        print(f"\n{len(device_df)} devices")
        print(device_df.columns)
        site_df = model_to_dataframe(Site)
        print(f"\n{len(site_df)} sites")
        print(device_df.columns)
        ticket_df = model_to_dataframe(Ticket)
        print(f"\n{len(ticket_df)} tickets")
        print(ticket_df.columns)
        task_df = model_to_dataframe(Task)
        print(f"\n{len(task_df)} tasks")
        print(task_df.columns)
        issue_df = model_to_dataframe(Issue)
        print(f"\n{len(issue_df)} issues")
        print(issue_df.columns)
        resolution_df = model_to_dataframe(Resolution)
        print(f"\n{len(resolution_df)} resolutions")
        print(resolution_df.columns)
    except OperationalError as e:
        print(f"OperationalError: {e}")

In [ ]:
import random

from app import create_app, db
from app.models import *


# 创建 Flask 应用实例
app = create_app()

def get_random_ticket_id():
    with app.app_context():
        # 获取所有 Ticket 的 ID
        ticket_ids = db.session.query(Ticket.id).all()

        if not ticket_ids:
            return "No tickets found"

        # 从中随机选择一个 ID
        random_ticket_id = random.choice(ticket_ids)[0]

        return random_ticket_id

with app.app_context():

    random_ticket_id = get_random_ticket_id()

    ticket = Ticket.query.get(random_ticket_id)

ticket

In [ ]:
import random, string

''.join(random.choices(string.ascii_uppercase, k=2)) + ''.join(random.choices(string.digits, k=10))

In [ ]:
import random

"A" + str(random.randint(10**9, 10**10 - 1))

In [ ]:
import json
import os

# 定义文件路径
file_path = os.path.join('data', 'samples', 'sites.json')

# 读取和解析 JSON 文件
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 遍历数据并打印详细信息
for client_name, sites in data.items():
    print(f"Company: {client_name}")
    for site_name, details in sites.items():
        print(f"    Name: {site_name}")
        print(f"    Address: {details['address']}")
        print(f"    ZIP: {details['zip']}")
        print(f"    Country: {details['country']}")
        print(f"    Latitude: {details['latitude']}")
        print(f"    Longitude: {details['longitude']}")
        print(f"    Devices:")
        for device_sn, info in details['devices'].items():
            print(f"        SN: {device_sn}")
            print(f"        Model: {info['model']}")
            print(f"        Material: {info['material']}")
            print(f"        Installed On: {info['install_on']}")
        print()

# 示例：访问特定位置的数据
#specific_location = data['Lidl']['Lidl-Elisenstr']['address']
#print(f"Specific Location: {specific_location}")

In [ ]:
import sys
import os

# 添加项目根目录到 sys.path
project_root = 'd:\\Projects\\Py-Projects\\Flask-Projects\\ticket-reporter'
sys.path.append(project_root)

from app.utilities import get_latest_csv, standardize_gsp

gsp_pattern = r"案例清单列表_(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})\.csv"
gsp_dir = 'D:\\Projects\\Py-Projects\\Flask-Projects\\ticket-handel-demo\\data\\gsp'

latest_gsp, _ = get_latest_csv(gsp_dir, gsp_pattern)

path_latest_gsp = os.path.join(gsp_dir, latest_gsp)

# 预设特定列的类型
dtype_spec = {18: 'str', 36: 'str', 37: 'str', 39: 'str', 49: 'str'}

gsp_df = pd.read_csv(path_latest_gsp,  dtype=dtype_spec)

In [ ]:
gsp_df.columns

In [ ]:
# | id | title | title_cn | create_on | ticket_type | description | status |
# | first_response | first_response_on | final_resolution | close_on |
def standardize() -> pd.DataFrame:


    gsp_df
    
    return gsp_df

In [ ]:
pip list --format=freeze > requirements.txt